<a href="https://colab.research.google.com/github/MedjialeuJordan/Machine-Learning-Projects/blob/main/MLP%2BLSTM%20Model%20on%20MNIST%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MNIST avec MLP et LSTM - Dropout + BatchNorm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Chargement des données ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=1000)

# --- MLP avec Dropout et BatchNorm ---
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.dropout1(F.relu(self.bn1(self.fc1(x))))
        x = self.dropout2(F.relu(self.bn2(self.fc2(x))))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# --- LSTM avec Dropout et BatchNorm ---
class LSTMClassifier(nn.Module):
    def __init__(self):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size=28, hidden_size=128, num_layers=1, batch_first=True)
        self.bn = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(128, 10)

    def forward(self, x):
        x = x.squeeze(1)  # (B, 1, 28, 28) -> (B, 28, 28)
        out, (hn, cn) = self.lstm(x)
        h_last = hn[-1]  # (B, 128)
        h_bn = self.bn(h_last)
        h_drop = self.dropout(h_bn)
        return F.log_softmax(self.fc(h_drop), dim=1)

# --- Entraînement et test ---
def train(model, optimizer, train_loader, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Test Accuracy: {accuracy:.2f}%")

# --- Choix du modele : MLP ou LSTM ---
model = MLP().to(device)  # ou LSTMClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# --- Entraînement sur 5 epochs ---
for epoch in range(1, 6):
    print(f"\n--- Epoch {epoch} ---")
    train(model, optimizer, train_loader, epoch)
    test(model, test_loader)



100%|██████████| 9.91M/9.91M [00:00<00:00, 59.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.64MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.7MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.02MB/s]



--- Epoch 1 ---
Test Accuracy: 96.65%

--- Epoch 2 ---
Test Accuracy: 97.34%

--- Epoch 3 ---
Test Accuracy: 97.73%

--- Epoch 4 ---
Test Accuracy: 97.74%

--- Epoch 5 ---
Test Accuracy: 97.86%
